In [138]:
import pandas as pd
import json
from pathlib import Path
from langdetect import detect

In [139]:
def load_json(path):
    with open(path, 'r') as file:
        loaded_data = json.load(file)
    
    #data = loaded_data['items']
    return loaded_data
    #return data

### create two paths to save .json

In [140]:
# file with repos up to 22196 stars
# file_name = '../data/df_repos_metadata_up_to_max.json'
# file with repos 0 to 22196 stars
file_name = '../data/df_repos_metadata_0_to_22196.json'

### get column names

In [141]:
data = load_json(path='../data/helper/help_columns.json')
# get keys of loaded_data as list
keys = list(data[0].keys())

In [142]:
len(keys)

82

### get paths

In [143]:
directory_path = Path("../data/raw_data")
all_files = [file.name for file in directory_path.iterdir() if file.is_file()]

In [144]:
all_files[0]

'checkpoint_350_multiple_github_repos.json'

### open all jsons and load repo metadata into dataframe

In [145]:
# create empty df with keys of loaded_data as columns
df_raw = pd.DataFrame(columns=keys)

In [146]:
for file in all_files:
    data = load_json(path=f'../data/raw_data/{file}')
    #  iterate through subdictionary in data and concatenate the content of the subdictionary to df_repos
    for repo in data:
        # create tmp df_repo for each repo
        df_tmp = pd.DataFrame(data=[repo], columns=keys)
        # concatenate df_repos with df_repo
        df_raw = pd.concat([df_raw, df_tmp], ignore_index=True)

/tmp/ipykernel_2618/143648634.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_raw = pd.concat([df_raw, df_tmp], ignore_index=True)


### clean df_repos and remove unnecessary columns

In [ ]:
# columns which are requiered for preprocessing and further steps
columns = [
    'id', 
    'name', 
    'full_name', 
    'html_url', 
    'description', 
    'url', 
    'labels_url', 
    'created_at', 
    'updated_at', 
    'pushed_at', 
    'size', 
    'stargazers_count', 
    'watchers_count', 
    'language', 
    'has_issues', 
    'has_projects', 
    'has_downloads', 
    'has_wiki', 
    'has_pages', 
    'has_discussions', 
    'forks_count', 
    'open_issues_count', 
    'license', 
    'allow_forking', 
    'topics', 
    'visibility', 
    'forks', 
    'open_issues', 
    'watchers', 
    'default_branch', 
    'score'
]

In [148]:
df_repos = df_raw

In [149]:
for k in keys:
    if k in columns:
        continue
    else:
        df_repos = df_repos.drop([k], axis=1)

In [150]:
df_repos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 591 non-null    object 
 1   name               591 non-null    object 
 2   full_name          591 non-null    object 
 3   html_url           591 non-null    object 
 4   description        579 non-null    object 
 5   url                591 non-null    object 
 6   labels_url         591 non-null    object 
 7   created_at         591 non-null    object 
 8   updated_at         591 non-null    object 
 9   pushed_at          591 non-null    object 
 10  size               591 non-null    object 
 11  stargazers_count   591 non-null    object 
 12  watchers_count     591 non-null    object 
 13  language           591 non-null    object 
 14  has_issues         591 non-null    object 
 15  has_projects       591 non-null    object 
 16  has_downloads      591 non

### save df_repos in json file

In [151]:
tmp_json = df_repos.to_json(orient='records', lines=False, force_ascii=False)

In [152]:
with open(file_name, 'w') as file:
    file.write(tmp_json)

### load json data for further analysis (test)

In [153]:
with open(file_name, 'r') as file:
    loaded_data = json.load(file)

In [154]:
test_df = pd.DataFrame(data=loaded_data)

### combine df's

In [155]:
directory_path = Path("../data")
all_dfs = [file.name for file in directory_path.iterdir() if file.is_file()]

In [156]:
all_dfs

['df_repos_metadata_star_up_to_max.json', 'df_repos_metadata_0_to_22196.json']

In [157]:
df_repos = pd.DataFrame()

In [158]:
with open('../data/df_repos_metadata_star_up_to_max.json', 'r') as file:
    loaded_data = json.load(file)
tmp_df = pd.DataFrame(data=loaded_data)

In [159]:
tmp_df.describe()

,id,size,stargazers_count,watchers_count,forks_count,open_issues_count,forks,open_issues,watchers,score
count,1.050000e+03,1.050000e+03,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.0
mean,2.842129e+08,1.316390e+05,21701.457143,21701.457143,3555.066667,375.716190,3555.066667,375.716190,21701.457143,1.0
std,2.800213e+08,4.986652e+05,29908.569921,29908.569921,5971.414570,1003.159613,5971.414570,1003.159613,29908.569921,0.0
min,2.655400e+04,7.000000e+00,7193.000000,7193.000000,108.000000,0.000000,108.000000,0.000000,7193.000000,1.0
25%,4.880424e+07,5.378500e+03,8829.500000,8829.500000,989.000000,56.000000,989.000000,56.000000,8829.500000,1.0
50%,1.615837e+08,2.407950e+04,12306.500000,12306.500000,1814.000000,139.500000,1814.000000,139.500000,12306.500000,1.0
75%,5.702785e+08,8.391375e+04,21947.250000,21947.250000,3423.000000,331.000000,3423.000000,331.000000,21947.250000,1.0
max,9.548733e+08,1.087097e+07,335520.000000,335520.000000,49242.000000,16073.000000,49242.000000,16073.000000,335520.000000,1.0


In [160]:
quantile_75 = tmp_df['stargazers_count'].describe().loc['75%']
quantile_75

np.float64(21947.25)

In [161]:
tmp_df = tmp_df[tmp_df['stargazers_count'] > quantile_75]

In [162]:
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 263 entries, 150 to 1019
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 263 non-null    int64  
 1   name               263 non-null    object 
 2   full_name          263 non-null    object 
 3   html_url           263 non-null    object 
 4   description        259 non-null    object 
 5   url                263 non-null    object 
 6   labels_url         263 non-null    object 
 7   created_at         263 non-null    object 
 8   updated_at         263 non-null    object 
 9   pushed_at          263 non-null    object 
 10  size               263 non-null    int64  
 11  stargazers_count   263 non-null    int64  
 12  watchers_count     263 non-null    int64  
 13  language           263 non-null    object 
 14  has_issues         263 non-null    bool   
 15  has_projects       263 non-null    bool   
 16  has_downloads      263 non-n

In [163]:
with open('../data/df_repos_metadata_0_to_22196.json', 'r') as file:
    loaded_data = json.load(file)
tmp_df1 = pd.DataFrame(data=loaded_data)

In [164]:
df_repos = pd.concat([tmp_df1, tmp_df])

In [165]:
df_repos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 854 entries, 0 to 1019
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 854 non-null    int64  
 1   name               854 non-null    object 
 2   full_name          854 non-null    object 
 3   html_url           854 non-null    object 
 4   description        838 non-null    object 
 5   url                854 non-null    object 
 6   labels_url         854 non-null    object 
 7   created_at         854 non-null    object 
 8   updated_at         854 non-null    object 
 9   pushed_at          854 non-null    object 
 10  size               854 non-null    int64  
 11  stargazers_count   854 non-null    int64  
 12  watchers_count     854 non-null    int64  
 13  language           854 non-null    object 
 14  has_issues         854 non-null    bool   
 15  has_projects       854 non-null    bool   
 16  has_downloads      854 non-nul

### filter df_repos for language

In [166]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "error"

In [167]:
df_repos['language_spoken'] = df_repos['description'].apply(detect_language)

In [168]:
len(df_repos[df_repos['language_spoken'] == 'en'])

748

### remove duplicate rows

In [169]:
# the behavior of the api is sometimes confusing
# i got some duplicates in the dataframe --> remove them
df_cleaned = df_repos.drop_duplicates(subset=['full_name'])


### save df_repos as .json for further analysis / work

In [170]:
tmp_json = df_cleaned.to_json(orient='records', lines=False, force_ascii=False)

In [171]:
with open('../data/df_repos_metadata.json', 'w') as file:
    file.write(tmp_json)